# ISOLATION FOREST FOR ANOMALY DETECTION 


Algoritmos de Isolation Forest para detecção de Anolamia na Folha de Pagamento - Rubricas de Auxilio Alimentacao
Importação do Modulo de Pré-processamento

In [4]:
import sys
sys.path.append("C:/Users/joaoc/Documents/MT/pagamento_servidores/src")
import process_servants as ps
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import pandas as pd
from janitor import clean_names
from datetime import datetime
from dateutil.relativedelta import relativedelta

## Função para pre-processamento dos dados
Utiliza-se funções importadas do arquivo process_servants.py, no qual foram pré-definidas algumas funções para o pré-processamento dos dados

In [6]:
def process_registration_data_rendim(file_path):
    """Process registration data from Excel file"""
    
    df = (
        pd.read_excel(file_path)
        .pipe(clean_names)
        .pipe(ps.convert_rendim_to_numeric)
        .pipe(ps.process_month_column)
        )    

    #Selecionar apesas as colunas necessárias para o modelo
    df_rendim = df[['cpf_servidor', 'mes', 'rendim']]
    #Se tiver valores com o CPF duplicado para o mesmo mês, somar os valores
    df_rendim = df_rendim.groupby(['cpf_servidor', 'mes'])['rendim'].sum().reset_index()

    return df_rendim

## Função para Leitura dos Dados

In [8]:
def processed_data(brute_data_file_path): 

    registration_df = process_registration_data_rendim(brute_data_file_path)
    
    return registration_df

Importação da Base de Dados

In [10]:
brute_file_path = 'C:/Users/joaoc/Documents/MT/pagamento_servidores/data/raw/servidores_mes_rubricas_alimentacao.xlsx'

In [12]:
df_alimentacao = processed_data(brute_file_path)

C:\Users\joaoc\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## Isolation Forest

In [14]:
def isolation_forest_alimentacao(df_alimentacao, select_month):
     # Filtrar dados do mês escolhido
    df_alimentacao_serie = df_alimentacao[df_alimentacao['mes'] == select_month].copy()
    
    # Remover a coluna 'mes'
    df_alimentacao_serie = df_alimentacao_serie.drop(columns='mes')
    
    # Definir índice e selecionar coluna
    df_alimentacao_serie = df_alimentacao_serie.set_index('cpf_servidor')[['rendim']]

    contamination = 0.5

    iso_forest = IsolationForest(
        contamination = contamination,
        random_state = 42
    )
    
    x_iso_forest = df_alimentacao_serie.values
    
    iso_forest.fit(x_iso_forest)
    
    anomalias = iso_forest.predict(x_iso_forest)
    
    df_alimentacao_serie['anomalia'] = anomalias
    
    return df_alimentacao_serie 
    

In [15]:
# Definir o mes para fazer a analise (Y, m, d=1)
i = 0
start_date = datetime(2025, 3, 1)
df_alimentacao_result_def = pd.DataFrame()
while i != 18:
    selected_date = start_date - relativedelta(months=i)
    print(selected_date.strftime('%Y-%m'))  # Format as needed
    i += 1
    #Run
    df_alimentacao_result = isolation_forest_alimentacao(df_alimentacao, selected_date)
    df_alimentacao_result = df_alimentacao_result[df_alimentacao_result['anomalia'] == -1]
    df_alimentacao_result['mes'] = selected_date
    df_alimentacao_result_def = pd.concat(
    [df_alimentacao_result_def, df_alimentacao_result], 
    axis=0  # Stack rows vertically (axis=0)
    )


2025-03
2025-02
2025-01
2024-12
2024-11
2024-10
2024-09
2024-08
2024-07
2024-06
2024-05
2024-04
2024-03
2024-02
2024-01
2023-12
2023-11
2023-10


In [16]:
df_alimentacao_result_def = df_alimentacao_result_def[df_alimentacao_result_def['rendim'] != 0]

In [18]:
df_alimentacao_result_def.to_excel('C:/Users/joaoc/Documents/MT/pagamento_servidores/data/processed/Alimentacao.xlsx')